# Standardised Approach for Counterparty Credit Risk

Abbreviations: <br >
- $VM$: Variation Margin
- $EAD$: Exposure At Default under SA
- $RC$: replacement cost (different for margined and unmargined)
- $PFE$: Potential Future Exposure
- $V$: Current Market Value of the Derivative in the netting set
- $C$: Net harcut value of the net collateral held
- $ICA$: Independent Collateral Amount
- $TH$: Positive threshold before the counterparty must send the bank collateral
- $MTA$: Minimum transfer amount
- $NICA$: Net independent Collateral Amount
- $MF_i^{type}$: Maturity Factor
- $M_i$: Maturity (i remaining maturity floored by 10 business days)
- $S_i$: Start
- $E_i$: end date
- $T_i$: Contractual Exercise Date
- ${SD}_i$: Supervisory Duration
- $\delta_i$: Supervisory delta Adjustments
- $MPOR_i$: Margin period of risk appropriate for the margin agreement containing the transaction i

Formulas: <br> 
- EAD = alpha * (RC + PFE) 
- RC (Margined) = max{V - C;TH + MTA - NICA; 0}
- PFE = multiplier * $AddOn^{aggregate}$
- multipiler = min{1; Floor + (1 - Floor) * exp ($\frac{V - C}{2* (1-Floor) * AddOn^{aggregate} }$)}
- $AddOn^{aggregate}$ = $\sum_{a} AddOn^a$
- ${SD}_i$ = $\frac {exp(-0.05*S_i)-exp(-0.05*Ei)} {0.05}$
- ${MF}_i^{unmargined}$ = $\sqrt{\frac {min\{M_i; 1 year\}}{1year}}$
- ${MF}_i^{(margined)}$ = $\frac {3} {2} \sqrt{\frac{MPOR_i}{1year}}$

<a href="https://www.bis.org/publ/bcbs279.pdf">PDF Reference</a>

In [1]:
import pandas as pd
import math
import numpy as np

In [2]:
# import reference table for supervisory parameter as supervisory_ref
supervisory_ref = pd.read_csv('supervisory_ref.csv')
supervisory_ref

,Asset Class,Subclass,Supervisory factor,Correlation
0,credit,AAA,0.0038,0.5
1,credit,AA,0.0038,0.5
2,credit,A,0.0042,0.5
3,credit,BBB,0.0054,0.5
4,credit,BB,0.0106,0.5
5,credit,B,0.0160,0.5
6,credit,CCC,0.0600,0.5
7,credit,IG,0.0038,0.8
8,credit,SG,0.0106,0.8


In [3]:
#rc(df) takes in a dataframe and compute the RC of a given file
def rc(df):
    v_c = df[['Market value (thousands)']].sum()
    r_c = max(int(v_c), 0)
    return r_c

In [4]:
# convert residual maturity to two lists, S and E by taking in
# three variables, emptyEi, emptySi and residual maturity
def residual_maturity_converter(Ei, Si, residual_maturity):
    for item in residual_maturity:
        item = str(item)
        idx = item.find("into")
        if (idx == -1): # S would be 0
            Si.append(0)
            end = int(item[:item.find("year") - 1])
            Ei.append(end)
        else:
            idx -= 1
            start = int(item[:idx])
            Si.append(start)
            idx += 5
            end = int(item[idx:item.find("year") - 1])
            Ei.append(end)
            
        for i in range(0,len(Ei)):
            Ei[i] = Ei[i] + Si[i]

In [5]:
#SD_generator takes in Si and Ei and generate the appropriate SD as a list
def SD_generator(Si, Ei, SD):
    if (len(Si) != len(Ei)):
        return "Error"
    for i in range(0, len(Si)):
        si = Si[i]
        ei = Ei[i]
        sd = (math.exp(-0.05*si) - math.exp(-0.05*ei)) / 0.05
        SD.append(sd)

In [6]:
# middle_df_generator(original_df) generates the middle_df for further cal
def middle_df_generator(original_df):
    data = {}
    hedging_set = original_df['Base currency'].tolist()
    nature_set = original_df['Nature'].tolist()
    length = len(nature_set)
    for i in range(0, length):
        indicator = nature_classifier(original_df, i)
        if indicator == 2:
            hedging_set[i] = 'CDS'
    Ei = []
    Si = []
    residual_maturity_converter(Ei, Si, original_df['Residual maturity'])
    SD = []
    SD_generator(Si, Ei, SD)
    Adj_notion = np.array(SD) * np.array(original_df['Notional (thousands)'].tolist())
    data.update({"Hedging set": hedging_set,
                 "Notional (thousands)": original_df['Notional (thousands)'].tolist(),
                "Si": Si, 
                "Ei":Ei,
                "SDi": SD,
                "Adjusted notional (thousands)": Adj_notion.tolist(),
                })
    return pd.DataFrame(data)

In [7]:
# nature_classifier(original_df, i) determine the category that the nature
# belongs to, 1 means interest rate, 2 means credit derivative
# 3 means commodity forrward contract
def nature_classifier(original_df, i):
    nature = str(original_df.at[i, 'Nature'])
    if (nature.find('swap') != -1):
        return 1
    elif nature.find('CDS') != -1:
        return 2
    else:
        return 3

In [8]:
# phi() takes in a number and calculate the phi of it
def phi(x):
    return (1.0 + math.erf(x / math.sqrt(2.0))) / 2.0

In [9]:
# supervisory_delta_generator() takes in two dataframe, the original one
# and the new generated one to modify and add a column on the 
# new generated dataframe
def supervisory_delta_generator(original_df, middle_df):
    (row_num, col_num) = original_df.shape
    sd_column = []
    for i in range(0, row_num):
        sd = None
        indicator = nature_classifier(original_df, i)
        if (indicator == 1):
        # This happens if it is a interest rate derivative
            fixed = (str(original_df.at[i, 'Pay Leg (*)']) == 'Fixed')
            option_idx = (str(original_df.at[i, 'Nature']).find("swaption"))
        
            if (fixed and option_idx == -1): # long but not option
                sd = 1
            elif (not fixed and option_idx == -1): # short but not option
                sd = -1
            else: # option
                symbol = 1
                if not fixed: #if it is call option
                    symbol = -1
                Ti = int(middle_df.at[i, 'Si'])
                sd = symbol * phi(symbol * 
                                  (math.log(0.06 / 0.05) + 
                                  0.5 * 0.5 ** 2 * Ti) / 
                                  (0.5 * math.sqrt(Ti)))
        
        if (indicator == 2): # if it is a credit derivative not an option
            #print('case two')
            long = str(original_df.at[i, 'Position']).find('buyer')
            if (long != -1): # it is a buyer so it would be long
                sd = 1
            else:
                sd = -1
        sd_column.append(sd)
        
    middle_df['Supervisory delta'] = sd_column

In [10]:
#time_bucket_classifer() classifies each transaction by time
def time_bucket_classifier(original_df, middle_df):
    classes = []
    for number in middle_df['Ei'].tolist():
        if number > 5:
            classes.append(3)
        elif number < 1:
            classes.append(1)
        else:
            classes.append(2)
    middle_df['Time Bucket'] = classes

    
# Hedging list generator
def hedging_list_generator(middle_df):
    series1 = middle_df['Adjusted notional (thousands)']
    series2 = middle_df['Supervisory delta']
    hedging_set = series1 * series2
    return hedging_set.tolist()

In [37]:
# EffectiveNotional generator for interest rate
def effectiveNotional(original_df, middle_df, group_name, indicator):
    entity_level_add_on_times_r = []
    entity_level_add_on_times_one_m = []
    
    
    hedging_set = hedging_list_generator(middle_df)
    (row_num, col_num) = original_df.shape
    en_list = [0, 0, 0]
    idx = 0
    for i in range(0,row_num): 
        if (str(middle_df.at[i, 'Hedging set']) == str(group_name) and indicator == 1):
            idx = middle_df.at[i, 'Time Bucket'] - 1
            en_list[idx] = hedging_set[i]
            
            
        elif (indicator == 2):
            symbol = 1
            if (original_df.at[i, 'Position'].find('seller') != -1):
                symbol = -1
            rating_ref = str(original_df.at[i, 'Rating reference entity'])
            print(rating_ref)
            if rating_ref == "Investment grade":
                rating_ref = 'IG'
            sf = pd.DataFrame(supervisory_ref[supervisory_ref['Subclass'] == rating_ref])['Supervisory factor'].tolist()[0]
            adj_notion = float(middle_df.at[i, 'Adjusted notional (thousands)'])
            addOn_temp = adj_notion * sf
            r = pd.DataFrame(supervisory_ref[supervisory_ref['Subclass'] == rating_ref])['Correlation'].tolist()[0]
            modified_r = 1 - r ** 2
            entity_level_aotr = addOn_temp * r * symbol
            entity_level_2tr = addOn_temp ** 2 * modified_r
            
            entity_level_add_on_times_r.append(entity_level_aotr)
            entity_level_add_on_times_one_m.append(entity_level_2tr)
            #print('indicator 2 ran')
            #print(en_list)
    print(entity_level_add_on_times_r)
    print(entity_level_add_on_times_one_m)
    if (indicator == 1):
        e_n = 0
        for item in en_list:
            e_n += item ** 2
    #print(e_n)
        e_n += 1.4 * (en_list[0]) * (en_list[1])
        e_n += 1.4 * (en_list[1]) * (en_list[2])
        e_n += 0.6 * (en_list[0]) * (en_list[2])
    #print(e_n)
        result = math.sqrt(e_n)
        return result
    elif (indicator == 2):
        
        addOn_credit = np.sum(entity_level_add_on_times_r) ** 2 + np.sum(entity_level_add_on_times_one_m)
        addOn_credit = math.sqrt(addOn_credit)
        print(addOn_credit)
        return addOn_credit

In [73]:
# addOn calculator
def addOn_generator(original_df, middle_df, supervisory_ref):
    # addOn_credit = []
    addOn = []
    unique_group = list(set(middle_df['Hedging set'].tolist()))
    print(unique_group)
    CDS = False
    interest_rate = False
    for group in unique_group:
        if (group.find("CDS") != -1):
            CDS = True
            print("CDS exists")
            break
    if CDS:
        for group in unique_group:
            if group.find("CDS") != -1:
                unique_group.remove(group)
                
        unique_group.append('CDS')
        
    effectiveNotionList_interest = []
    entity_level_add_on_times_r = []
    # = []
    entity_level_add_on_times_one_m = []
    
    i = 0
    for hedging_set in unique_group:
        indicator = nature_classifier(original_df, i)
        if indicator == 1: # deal with cases of interest rate
            effectiveNotionList_interest.append(effectiveNotional(original_df, middle_df, hedging_set, indicator))
            interest_rate = True;
            
        elif indicator == 2: #deal with cases of CDS
            addOn.append(effectiveNotional(original_df, middle_df,hedging_set, 2))
            CDS = True
            
            
        i += 1
    #print(effectiveNotionList)
   
    if (interest_rate):
        newList = 0.005 * np.array(effectiveNotionList_interest)
        #print("Error here?")
    #print(newList)
        addOn_interest = np.sum(newList)
        addOn.append(addOn_interest)
    
    # we now deal with the credit part
    if (CDS):
        print("CDS IS TRUE")
        
        #addOn.append(addOn_credit)
    return (np.sum(addOn))

In [74]:
# find the multipler for interest rate EAD
def multiplier_finder(original_df, middle_df):
    addOn = addOn_generator(original_df, middle_df, supervisory_ref)
    floor = 0.05
    v_c = original_df[['Market value (thousands)']].sum()
    denominator = 2 * (1 - floor) * addOn
    alternative = floor + (1 - floor) * math.exp(v_c / denominator)
    multiplier = min(1, alternative)
    return multiplier

In [75]:
# EAD for interest rate
def EAD_generator(original_df, middle_df, supervisory_ref):
    r_c = rc(original_df)
    print("RC:", r_c)
    multiplier = multiplier_finder(original_df, middle_df)
    print('multiplier:', multiplier)
    addon_val = addOn_generator(original_df, middle_df, supervisory_ref)
    print("addOn Value", addon_val)
    
    return 1.4 * (r_c + multiplier * addon_val)

# Example 1

In [76]:
example_1_df = pd.read_csv('example_1_input.csv')
middle_df = middle_df_generator(example_1_df)
supervisory_delta_generator(example_1_df, middle_df)
time_bucket_classifier(example_1_df, middle_df)
hedging_list_generator(middle_df)
addOn_generator(example_1_df, middle_df,supervisory_ref)
example_1_df

['USD', 'EUR']
[]
[]
[]
[]


,Trade #,Nature,Residual maturity,Base currency,Notional (thousands),Pay Leg (*),Receive Leg (*),Market value (thousands)
0,1,Interest rate swap,10 years,USD,10000,Fixed,Floating,30
1,2,Interest rate swap,4 years,USD,10000,Floating,Fixed,-20
2,3,European swaption,1 into 10 years,EUR,5000,Floating,Fixed,50


In [77]:
EAD_generator(example_1_df, middle_df, supervisory_ref)

RC: 60
['USD', 'EUR']
[]
[]
[]
[]
multiplier: 1
['USD', 'EUR']
[]
[]
[]
[]
addOn Value 346.7643863838185


569.4701409373458

In [78]:
middle_df

,Hedging set,Notional (thousands),Si,Ei,SDi,Adjusted notional (thousands),Supervisory delta,Time Bucket
0,USD,10000,0,10,7.869387,78693.868057,1.000000,3
1,USD,10000,0,4,3.625385,36253.849384,-1.000000,2
2,EUR,5000,1,11,7.485592,37427.961412,-0.269395,3


# Example 2

In [61]:
example_2_input = pd.read_csv('example_2_input.csv')
example_2_input

,Trade #,Nature,Reference entity / index name,Rating reference entity,Residual maturity,Base currency,Notional (thousands),Position,Market value (thousands)
0,1,Single-name CDS,Firm A,AA,3 years,USD,10000,Protection buyer,20
1,2,Single-name CDS,Firm B,BBB,6 years,EUR,10000,Protection seller,-40
2,3,CDS index,CDX.IG 5y,Investment grade,5 years,USD,10000,Protection buyer,0


In [62]:
ex_2_middle = middle_df_generator(example_2_input)
ex_2_middle

,Hedging set,Notional (thousands),Si,Ei,SDi,Adjusted notional (thousands)
0,CDS,10000,0,3,2.785840,27858.404715
1,CDS,10000,0,6,5.183636,51836.355864
2,CDS,10000,0,5,4.423984,44239.843386


In [63]:
supervisory_delta_generator(example_2_input, ex_2_middle)
ex_2_middle

,Hedging set,Notional (thousands),Si,Ei,SDi,Adjusted notional (thousands),Supervisory delta
0,CDS,10000,0,3,2.785840,27858.404715,1
1,CDS,10000,0,6,5.183636,51836.355864,-1
2,CDS,10000,0,5,4.423984,44239.843386,1


In [64]:
addOn_generator(example_2_input, ex_2_middle,supervisory_ref)

['CDS']
CDS exists
AA
BBB
Investment grade
[52.93096895847803, -139.95816083187233, 134.4891238925858]
[8405.062424650096, 58764.86035032073, 10174.12000053485]
282.1288318596666
CDS IS TRUE


282

In [66]:
EAD_generator(example_2_input, ex_2_middle,supervisory_ref)

['CDS']
CDS exists
AA
BBB
Investment grade
[52.93096895847803, -139.95816083187233, 134.4891238925858]
[8405.062424650096, 58764.86035032073, 10174.12000053485]
282.1288318596666
CDS IS TRUE
['CDS']
CDS exists
AA
BBB
Investment grade
[52.93096895847803, -139.95816083187233, 134.4891238925858]
[8405.062424650096, 58764.86035032073, 10174.12000053485]
282.1288318596666
CDS IS TRUE


381.05807053270433

# Example 4

In [67]:
example_4_input = pd.read_csv('example_4_input.csv')
example_4_input

,,,,,,,,,,,example_4_input
Trade #,Nature,Residual maturity,Base currency,Notional (thousands),Pay Leg (*),Receive Leg (*),Market value (thousands),Reference entity / index name,Rating reference entity,Position,NaN
1,Interest rate swap,10 years,USD,10000,Fixed,Floating,30,NaN,NaN,NaN,NaN
2,Interest rate swap,4 years,USD,10000,Floating,Fixed,-20,NaN,NaN,NaN,NaN
3,European swaption,1 into 10 years,EUR,5000,Floating,Fixed,50,NaN,NaN,NaN,NaN
4,Single-name CDS,3 years,USD,10000,NaN,NaN,20,Firm A,AA,Protection buyer,NaN
5,Single-name CDS,6 years,EUR,10000,NaN,NaN,-40,Firm B,BBB,Protection seller,NaN
6,CDS index,5 years,USD,10000,NaN,NaN,0,CDX.IG 5y,Investment grade,Protection buyer,NaN
